# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path) 

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rawson,AR,-43.3002,-65.1023,67.23,57,100,12.95,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,60.91,69,0,13.00,clear sky
2,2,Hilo,US,19.7297,-155.0900,76.12,85,75,12.66,moderate rain
3,3,Chinsali,ZM,-10.5414,32.0816,65.46,81,100,5.66,light rain
4,4,Muros,ES,42.7762,-9.0603,60.10,94,91,7.61,overcast clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,67.55,89,38,5.01,scattered clouds
6,6,Jamestown,US,42.0970,-79.2353,56.79,27,0,12.66,clear sky
7,7,Tateyama,JP,34.9833,139.8667,69.73,55,0,19.86,clear sky
8,8,Mahebourg,MU,-20.4081,57.7000,72.25,83,20,0.00,few clouds
9,9,Bredasdorp,ZA,-34.5322,20.0403,56.89,68,35,8.86,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                    (city_data_df["Max Temp"] > min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rawson,AR,-43.3002,-65.1023,67.23,57,100,12.95,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,76.12,85,75,12.66,moderate rain
3,3,Chinsali,ZM,-10.5414,32.0816,65.46,81,100,5.66,light rain
5,5,Puerto Ayora,EC,-0.7393,-90.3518,67.55,89,38,5.01,scattered clouds
7,7,Tateyama,JP,34.9833,139.8667,69.73,55,0,19.86,clear sky
8,8,Mahebourg,MU,-20.4081,57.7000,72.25,83,20,0.00,few clouds
13,13,Vaini,TO,-21.2000,-175.2000,78.96,89,75,8.05,broken clouds
19,19,Oranjestad,AW,12.5240,-70.0270,80.55,96,20,2.30,few clouds
21,21,Hithadhoo,MV,-0.6000,73.0833,81.84,73,85,13.51,overcast clouds
22,22,Fukue,JP,32.6881,128.8419,68.36,83,100,2.30,moderate rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()
# Display sample data
clean_travel_cities.count()

City_ID        278
City           278
Country        278
Lat            278
Lng            278
Max Temp       278
Humidity       278
Cloudiness     278
Wind Speed     278
Description    278
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng
0,Rawson,AR,67.23,overcast clouds,-43.3002,-65.1023
2,Hilo,US,76.12,moderate rain,19.7297,-155.0900
3,Chinsali,ZM,65.46,light rain,-10.5414,32.0816
5,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518
7,Tateyama,JP,69.73,clear sky,34.9833,139.8667


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rawson,AR,67.23,overcast clouds,-43.3002,-65.1023,
2,Hilo,US,76.12,moderate rain,19.7297,-155.0900,
3,Chinsali,ZM,65.46,light rain,-10.5414,32.0816,
5,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518,
7,Tateyama,JP,69.73,clear sky,34.9833,139.8667,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params = params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("Hotel assignment completed")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel assignment completed


In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rawson,AR,67.23,overcast clouds,-43.3002,-65.1023,Hosteria Sampedro
2,Hilo,US,76.12,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Chinsali,ZM,65.46,light rain,-10.5414,32.0816,Chisakatu Guest House
5,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Tateyama,JP,69.73,clear sky,34.9833,139.8667,Tateyama Seaside Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace = True)
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rawson,AR,67.23,overcast clouds,-43.3002,-65.1023,Hosteria Sampedro
2,Hilo,US,76.12,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Chinsali,ZM,65.46,light rain,-10.5414,32.0816,Chisakatu Guest House
5,Puerto Ayora,EC,67.55,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Tateyama,JP,69.73,clear sky,34.9833,139.8667,Tateyama Seaside Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"
# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv("./" + output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, 
# and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Hosteria Sampedro</dd>\n<dt>City</dt><dd>Rawson</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 67.23 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>moderate rain and 76.12 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Chisakatu Guest House</dd>\n<dt>City</dt><dd>Chinsali</dd>\n<dt>Country</dt><dd>ZM</dd>\n<dt>Current Weather</dt><dd>light rain and 65.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 67.55 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Tateyama Seaside Hotel</dd>\n<dt>City</dt><dd>Tateyama</dd>\n<dt>Country</dt><dd>JP</dd>\n<dt>Current Weather</dt><dd>clear sky and 69.73 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shandra

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-43.3002,-65.1023
2,19.7297,-155.0900
3,-10.5414,32.0816
5,-0.7393,-90.3518
7,34.9833,139.8667
8,-20.4081,57.7000
13,-21.2000,-175.2000
19,12.5240,-70.0270
21,-0.6000,73.0833
22,32.6881,128.8419


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info) # creating markers for each city

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))